Bitswan Gitops Webhook Pipeline
-------------------------------

This pipeline allows you to easilly deploy Bitswan pipelines to any server. You just need to POST to the webhook in your CI and your pipelines will be running in no time.

To install bitswan you need to initialize a git repository on your server/the place you want to run your bitswan installation and run the bitswan-pre docker image with the following ENV vars set:

- `BS_WEBHOOK_PORT` - defaults to 8080
- `BS_WEBHOOK_SECRET` - A secret that will be added to your webhook URL. To deploy post `{"action": "deploy-git"}` to the URL `https://localhost:<BS_WEBHOOK_PORT>/?secret=<BS_WEBHOOK_SECRET>`
- `BS_BITSWAN_DIR` - The directory where your `bitswan.yaml` file resides. Should be in a checked out git repository. Defaults to `/mnt/repo/bitswan`.

Simply adding the following curl command to your CI/CD pipeline should be enough to automatically deploy your data machines:

```
curl -X POST "https://<WEBHOOK_URL>:<BS_WEBHOOK_PORT>/?secret=<BS_WEBHOOK_SECRET>" -d '{"action": "deploy-git"}'
```

In [2]:
from bspump.jupyter import *
import bspump.http.web.source
import bspump.common
import time
import json
import os
import asyncio
import yaml
import datetime
from copy import deepcopy

In [3]:
new_pipeline("BitswanGitopsWebHookPipeline")

In [4]:
@register_source
def source(app, pipeline):
   return bspump.http.web.source.WebHookSource(
       app,
       pipeline,
       config = {
           "port": int(os.environ.get("BS_WEBHOOK_PORT", 8080)),
           "path": "/",
           "secret_qparam": os.environ.get("BS_WEBHOOK_SECRET", "not-secure")
       })

In [5]:
sample_events([
    b"""{"action": "deploy-git"}"""
])

In [6]:
@step
def parse_json(event):
    return json.loads(event)

{'action': 'deploy-git'}


In [7]:
@step
def get_bitswan_dir(event):
    event["bitswan_dir"] = os.environ.get("BS_BITSWAN_DIR", "/mnt/repo/bitswan")
    return event

{'action': 'deploy-git', 'bitswan_dir': '/mnt/repo/bitswan'}


In [8]:
sample_events([{'action': 'deploy-git', "start-ides": ["bitswan-ingress"], "bitswan_dir":"/home/timothy/pr/libertyaces/product/poc-pipelines"}])

In [9]:
@async_step
async def git_pull(inject, event):
    bitswan_dir = event["bitswan_dir"]
    print("Pulling in new changes.")
    git_repo_root = os.path.abspath(bitswan_dir)  # Convert to absolute path

    while git_repo_root != os.path.dirname(git_repo_root):  # Check until the root of the file system
        if os.path.isdir(os.path.join(git_repo_root, '.git')):
            break  # Return the path if .git directory is found
        git_repo_root = os.path.dirname(git_repo_root)  # Move up one directory level

    await asyncio.create_subprocess_exec("git", "config", "--global", "--add", "safe.directory", git_repo_root) 
    await asyncio.create_subprocess_exec("git", "config", "pull.rebase", "false", cwd=bitswan_dir) 
    await asyncio.create_subprocess_exec("git", "pull", cwd=bitswan_dir)
    print("Done pulling in changes.")
    await inject(event)

Pulling in new changes.
Done pulling in changes.
{'action': 'deploy-git', 'start-ides': ['bitswan-ingress'], 'bitswan_dir': '/home/timothy/pr/libertyaces/product/poc-pipelines'}


In [10]:
@step
def load_bitswan_yaml(event):
    event["yaml"] = yaml.full_load(open(os.path.join(event["bitswan_dir"], "bitswan.yaml")))
    return event

{'action': 'deploy-git', 'start-ides': ['bitswan-ingress'], 'bitswan_dir': '/home/timothy/pr/libertyaces/product/poc-pipelines', 'yaml': {'env-dir': '/etc/bitswan-secrets/', 'data-machines': {'bitswan-ingress': {'enabled': False}}}}


In [11]:
@step
def generate_docker_compose(event):
    bitswan_dir = event["bitswan_dir"]
    bitswan_yaml = event["yaml"]
    dc = {
        "version": "3",
        "services": {},
        "networks": {}
    }
    for network in bitswan_yaml.get("default-networks", []):
        dc["networks"][network] = {"external": True}
    for deployment_id, conf in bitswan_yaml["data-machines"].items():
        if conf is None:
            conf = {}
        entry = {}
        entry["environment"] = {"DEPLOYMENT_ID": deployment_id}
        if "env-dir" in bitswan_yaml:
           env_file = os.path.join(bitswan_yaml["env-dir"], deployment_id)
           if os.path.exists(env_file):
               entry["env_file"] = [env_file]
        if "default-networks" in bitswan_yaml:
            entry["networks"] = bitswan_yaml["default-networks"].copy()
        source = conf.get("source", deployment_id)
        data_machine_dir = os.path.join(bitswan_dir, source)
        dockerfile_path = "Dockerfile"
        entry["build"] = {
            "dockerfile": os.path.join(data_machine_dir, dockerfile_path),
            "context": bitswan_dir,
            "args": {
                "DATA_MACHINE_SOURCE_PATH": source,
            }
        }
        passthroughs = ["volumes", "network_mode", "ports", "restart", "devices", "container_name", ]
        for passthrough in passthroughs:
            if passthrough in conf:
              entry[passthrough] = conf[passthrough]
        if conf.get("enabled", True):
            dc["services"][deployment_id] = entry
        if deployment_id in event.get("start-ides", []):
            ide_entry = deepcopy(entry)
            ide_entry["entrypoint"] = "/start-ide.sh"
            ide_entry["deploy"] = ide_entry.get("deploy", {})
            ide_entry["deploy"]["labels"] = ide_entry["deploy"].get("labels", {})
            ide_entry["deploy"]["labels"]["space.bitswan.mode"] = "IDE"
            ide_entry["volumes"] = ide_entry.get("volumes", [])
            ide_entry["volumes"].append("/src:/mnt")
            dc["services"][deployment_id+"__ide__"] = ide_entry
    dc_yaml = yaml.dump(dc)
    print(dc_yaml)
    return (bitswan_dir, dc_yaml)

networks: {}
services:
  bitswan-ingress__ide__:
    build:
      args:
        DATA_MACHINE_SOURCE_PATH: bitswan-ingress
      context: /home/timothy/pr/libertyaces/product/poc-pipelines
      dockerfile: /home/timothy/pr/libertyaces/product/poc-pipelines/bitswan-ingress/Dockerfile
    deploy:
      labels:
        space.bitswan.mode: IDE
    entrypoint: /start-ide.sh
    environment:
      DEPLOYMENT_ID: bitswan-ingress
    volumes:
    - /src:/mnt
version: '3'

('/home/timothy/pr/libertyaces/product/poc-pipelines', "networks: {}\nservices:\n  bitswan-ingress__ide__:\n    build:\n      args:\n        DATA_MACHINE_SOURCE_PATH: bitswan-ingress\n      context: /home/timothy/pr/libertyaces/product/poc-pipelines\n      dockerfile: /home/timothy/pr/libertyaces/product/poc-pipelines/bitswan-ingress/Dockerfile\n    deploy:\n      labels:\n        space.bitswan.mode: IDE\n    entrypoint: /start-ide.sh\n    environment:\n      DEPLOYMENT_ID: bitswan-ingress\n    volumes:\n    - /src:/mnt\nvers

In [12]:
@async_step
async def docker_compose_up_daemon(inject, event):
    (bitswan_dir, docker_compose_yaml) = event
    cmd = ["docker-compose", "-f", "/dev/stdin", "build", "--pull"]

    # Create a subprocess with stdin pipe
    proc = await asyncio.create_subprocess_exec(
        *cmd, 
        stdin=asyncio.subprocess.PIPE, 
        stdout=asyncio.subprocess.PIPE, 
        stderr=asyncio.subprocess.PIPE,
        cwd=bitswan_dir
    )

    # Send docker_compose_yaml as input to the process and wait for completion
    stdout, stderr = await proc.communicate(input=docker_compose_yaml.encode())

    build_result = {
        "cmd": cmd,
        "stdout": stdout.decode("utf-8"),
        "stderr": stderr.decode("utf-8"),
        "returncode": proc.returncode,
    }
    
    cmd = ["docker-compose", "-f", "/dev/stdin", "up", "-d"]

    # Create a subprocess with stdin pipe
    proc = await asyncio.create_subprocess_exec(
        *cmd, 
        stdin=asyncio.subprocess.PIPE, 
        stdout=asyncio.subprocess.PIPE, 
        stderr=asyncio.subprocess.PIPE,
        cwd=bitswan_dir
    )

    # Send docker_compose_yaml as input to the process and wait for completion
    stdout, stderr = await proc.communicate(input=docker_compose_yaml.encode())

    up_result = {
        "cmd": cmd,
        "stdout": stdout.decode("utf-8"),
        "stderr": stderr.decode("utf-8"),
        "returncode": proc.returncode,
    }
    event = {
        "@timestamp": datetime.datetime.now().timestamp(),
        "local-time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "build": build_result,
        "up": up_result
    }
    await inject(event)

{'@timestamp': 1706896896.047645, 'local-time': '2024-02-02 19:01:36', 'build': {'cmd': ['docker-compose', '-f', '/dev/stdin', 'build', '--pull'], 'stdout': '', 'stderr': 'invalid tag "dev-bitswan-ingress__ide__": invalid reference format\n', 'returncode': 17}, 'up': {'cmd': ['docker-compose', '-f', '/dev/stdin', 'up', '-d'], 'stdout': '', 'stderr': 'Error response from daemon: no such image: dev-bitswan-ingress__ide__: invalid reference format\n', 'returncode': 1}}


Your configuration specifies to merge with the ref 'refs/heads/main'
from the remote, but no such ref was fetched.


In [ ]:
@step
def serialize_yaml(event):
    return yaml.dump(event)

In [ ]:
@register_sink
def init_sink(app, pipeline):
    return bspump.common.PrintSink(app, pipeline)

In [ ]:
end_pipeline()